Este notebook toma los archivos json y los procesa con la librería datasets de Huggingface. Durante el procesado, el texto y las entidades se fusionan en un mismo string que será utilizado para el fine-tuning de GPT-J.

In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.3 MB/s eta 0:00:00


In [4]:
import json
from datasets import Dataset

def load_custom_dataset(path):
    with open(path, 'r') as f:
        data = json.load(f)
    texts = [item['text'] for item in data]
    entities = [item['entities'] for item in data]
    return Dataset.from_dict({'text': texts, 'entities': entities})

dataset = load_custom_dataset('/content/drive/MyDrive/hackathon_podcast/data/NER/json_entity/output_0.json')

In [5]:
print(dataset['text'][0])  # prints the first text in the dataset
print(dataset['entities'][0][0])  # prints the first list of entities in the dataset

Yo hoy voy a hablar de mujeres en el mundo del arte, porque me ha leído un libro fantástico que se llama Historia del arte sin hombres, de Katie Hesel.

{'class': 'people', 'text': 'Katie Hesel'}


In [6]:
def form_training_prompts(example):
    text = example["text"]
    text = text[:-1]
    entities_list = example["entities"]
    entities = ""
    for entity in entities_list:
        entities = entities + '(' + entity['class'] + ', ' + entity['text'] + '), '
    entities = entities[:-2]

    example[
        "prompt"
    ] = f"text: {text} entities: {entities}<|endoftext|>"
    return example

In [7]:
dataset = dataset.map(
    form_training_prompts,
    remove_columns=["text", "entities"],
    load_from_cache_file=False,
    desc="Generating text prompt",
)

Generating text prompt:   0%|          | 0/37 [00:00<?, ? examples/s]

In [8]:
# shows first textual prompt
print(dataset[0])
print(dataset[1])
print(dataset[2])
print(dataset[3])

{'prompt': 'text: Yo hoy voy a hablar de mujeres en el mundo del arte, porque me ha leído un libro fantástico que se llama Historia del arte sin hombres, de Katie Hesel. entities: (people, Katie Hesel), (books, Historia del arte sin hombres)<|endoftext|>'}
{'prompt': 'text: Lo que hace Katie Hesel en este libro es el resultado de las exposiciones y decir que otra vez no hay mujeres en esta exposición, como todo el tiempo. entities: (people, Katie Hesel), (books, este libro)<|endoftext|>'}
{'prompt': 'text: Por supuesto, hace también en la introducción un recuento histórico, las mujeres artistas son el 1% por ejemplo de la National Gallery de Londres, el 1% y el público general de su país, el Reino Unido, no pueden nombrar a más de tres artistas mujeres. entities: (organizations, National Gallery de Londres), (places, Reino Unido)<|endoftext|>'}
{'prompt': 'text: Hesel lo que hace en este libro es utilizar el título del libro más famoso y más reputado de la historia del arte, el de Gomb